# Second tutorial

This tutorial is inspired by the official **[set of tutorials](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb)** that you have to study.  

# MuJoCo basics

As always first we need to upload all needed libraries. Here we are going to use mujoco and mujoco-python-viewer. You can use the default viewer that I used in the previous example, but I like the custom one. 

In [1]:
import mujoco
import mujoco_viewer

Here I have created some weird model written in MJCF and saved as xml file. Since I did not save it as a separate file, we are using `from_xml_string` method to create MuJoCo model entity. 

In [2]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
      <geom name="blue_cylinder" type="cylinder" pos="0.1 -0.2 0.0" size="0.1 0.2" rgba="0 0 1 1"/>
    </body>
  </worldbody>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(xml)

The `xml` string is written in MuJoCo's [MJCF](http://www.mujoco.org/book/modeling.html), which is an [XML](https://en.wikipedia.org/wiki/XML#Key_terminology)-based modeling language.
  - The only required element is `<mujoco>`. The smallest valid MJCF model is `<mujoco/>` which is a completely empty model.
  - All physical elements live inside the `<worldbody>` which is always the top-level body and constitutes the global origin in Cartesian coordinates.
  - We define two geoms in the world named `red_box` and `green_sphere`.
  - **Question:** The `red_box` has no position, the `green_sphere` has no type, why is that?
    - **Answer:** MJCF attributes have *default values*. The default position is `0 0 0`, the default geom type is `sphere`. The MJCF language is described in the documentation's [XML Reference chapter](https://mujoco.readthedocs.io/en/latest/XMLreference.html).

The `from_xml_string()` method invokes the model compiler, which creates a binary `mjModel` instance.

## mjModel

MuJoCo's `mjModel`, contains the *model description*, i.e., all quantities which *do not change over time*. The complete description of `mjModel` can be found at the end of the header file [`mjmodel.h`](https://github.com/deepmind/mujoco/blob/main/include/mujoco/mjmodel.h). Note that the header files contain short, useful inline comments, describing each field.

Examples of quantities that can be found in `mjModel` are `ngeom`, the number of geoms in the scene and `geom_rgba`, their respective colors:

In [ ]:
model.ngeom

In [ ]:
model.geom_rgba

## Named access

The MuJoCo Python bindings provide convenient [accessors](https://mujoco.readthedocs.io/en/latest/python.html#named-access) using names. Calling the `model.geom()` accessor without a name string generates a convenient error that tells us what the valid names are.

In [ ]:
try:
  model.geom()
except KeyError as e:
  print(e)

Calling the named accessor without specifying a property will tell us what all the valid properties are:

In [ ]:
model.geom('green_sphere')

Let's read the `green_sphere`'s rgba values:

In [ ]:
model.geom('green_sphere').rgba

This functionality is a convenience shortcut for MuJoCo's [`mj_name2id`](https://mujoco.readthedocs.io/en/latest/APIreference.html?highlight=mj_name2id#mj-name2id) function:

In [ ]:
id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'green_sphere')
model.geom_rgba[id, :]

Similarly, the read-only `id` and `name` properties can be used to convert from id to name and back:

In [ ]:
print('id of "green_sphere": ', model.geom('green_sphere').id)
print('name of geom 1: ', model.geom(1).name)
print('name of body 0: ', model.body(0).name)

Note that the 0th body is always the `world`. It cannot be renamed.

The `id` and `name` attributes are useful in Python comprehensions:

In [ ]:
[model.geom(i).name for i in range(model.ngeom)]

In [ ]:
print(f"\nThe number of geoms is {model.ngeom}. Geom colors are {model.geom_rgba}")
print(model.geom("green_sphere").pos)

id_1 = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'green_sphere')
id_2 = model.geom('red_box').id
id_2_2 = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'red_box')
print(f"\nID of the red_box is {id_2}")
print(f"Name of the object with ID={id_1} is {model.geom(id_1).name}")
print(f"Name of the object with ID={id_2} is {model.geom(id_2).name}")
print(f"Pos of the object with ID={id_2} is {model.geom(id_2).pos}")
print(f"Quat of the object with ID={id_2_2} is {model.geom(id_2_2).quat}")

## `mjData`
`mjData` contains the *state* and quantities that depend on it. The state is made up of time, [generalized](https://en.wikipedia.org/wiki/Generalized_coordinates) positions and generalized velocities. These are respectively `data.time`, `data.qpos` and `data.qvel`. In order to make a new `mjData`, all we need is our `mjModel`

In [12]:
data = mujoco.MjData(model)

`mjData` also contains *functions of the state*, for example the Cartesian positions of objects in the world frame. The (x, y, z) positions of our two geoms are in `data.geom_xpos`:

In [ ]:
data.geom_xpos

Wait, why are both of our geoms at the origin? Didn't we offset the green sphere? The answer is that derived quantities in `mjData` need to be explicitly propagated (see [below](#scrollTo=QY1gpms1HXeN)). In our case, the minimal required function is [`mj_kinematics`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-kinematics), which computes global Cartesian poses for all objects (excluding cameras and lights).

In [ ]:
mujoco.mj_kinematics(model, data)
print('raw access:\n', data.geom_xpos)

# MjData also supports named access:
print('\nnamed access:\n', data.geom('green_sphere').xpos)

In [ ]:
print(f"\nTime is {data.time}")
print(f"Pos of all geoms is {data.geom_xpos}")
print(f"q_pos of all geoms is {data.qpos}")
print(f"q_vel of all geoms is {data.qvel}")

# Options

Few code lines to show how you can operate on options. 

In [ ]:
model.opt.gravity = [0, 0, 9.81]
print(f"Timestep is {model.opt.timestep}")
print(f"Gravity is {model.opt.gravity}")

print('Total number of DoFs in the model:', model.nv)
print('Generalized positions:', data.qpos)
print('Generalized velocities:', data.qvel)

# Basic rendering, simulation, and animation

In order to render we'll need to instantiate a `Renderer` object and call its `render` method.

We'll also reload our model to make the colab's sections independent.

In [17]:
viewer = mujoco_viewer.MujocoViewer(model, data, title="tutorial", width=1920, height=1080)
while viewer.is_alive:
    mujoco.mj_step(model, data)

    viewer.render()

viewer.close()   